In [13]:
# Дополнительное задание #

In [22]:
# библиотеки и входные данные #
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# РАБОТАЕТ!!! #

# тест #
# на выходе 0, если 0 > 1 #
# на выходе 1, если 1 > 0 #
x = np.array(([0,0,1],[0,1,1],[1,0,1],[1,1,0]), dtype=float)
y = np.array(([0],[1],[1],[1]), dtype=float)

# нормальные входные данные #
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_iris
dataset = load_breast_cancer()
dataset_iris = load_iris()
X = dataset.data
Y = dataset.target
X_iris = dataset_iris.data
Y_iris = dataset_iris.target

In [15]:
Y.shape  = (Y.shape[0], 1)
Y_iris.shape  = (Y_iris.shape[0], 1)

In [16]:
# Разделение набора данных #
scaler = StandardScaler()
test_size = 0.25
seed = 42

# рак #
X_train_class, X_test_class, Y_train_class, Y_test_class     = train_test_split(X[:,:29], Y, test_size=test_size, random_state=seed)
X_train_regres, X_test_regres, Y_train_regres, Y_test_regres = train_test_split(X[:,:29], X[:,-1], test_size=test_size, random_state=seed)
# Попытка в нормализацию данных #
X_train_class_scaled = scaler.fit_transform(X_train_class)
X_test_class_scaled = scaler.fit_transform(X_test_class)

X_train_regres_scaled = scaler.fit_transform(X_train_regres)
X_test_regres_scaled = scaler.fit_transform(X_test_regres)


# ирис #
X_train_class_iris, X_test_class_iris, Y_train_class_iris, Y_test_class_iris     = train_test_split(X_iris[:,:3], Y_iris, test_size=test_size, random_state=seed)
X_train_regres_iris, X_test_regres_iris, Y_train_regres_iris, Y_test_regres_iris = train_test_split(X_iris[:,:3], X_iris[:,-1], test_size=test_size, random_state=seed)
# Попытка в нормализацию данных #
X_train_class_iris_scaled = scaler.fit_transform(X_train_class_iris)
X_test_class_iris_scaled = scaler.fit_transform(X_test_class_iris)

X_train_regres_iris_scaled = scaler.fit_transform(X_train_regres_iris)
X_test_regres_iris_scaled = scaler.fit_transform(X_test_regres_iris)

In [17]:
# Выравнивание #
Y_train_class.shape  = (Y_train_class.shape[0], 1)
Y_test_class.shape   = (Y_test_class.shape[0], 1)
Y_train_regres.shape = (Y_train_regres.shape[0], 1)
Y_test_regres.shape  = (Y_test_regres.shape[0], 1)

Y_train_class_iris.shape  = (Y_train_class_iris.shape[0], 1)
Y_test_class_iris.shape   = (Y_test_class_iris.shape[0], 1)
Y_train_regres_iris.shape = (Y_train_regres_iris.shape[0], 1)
Y_test_regres_iris.shape  = (Y_test_regres_iris.shape[0], 1)

In [18]:
# Функции активации #
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    arr = x > 0
    return arr.astype(float) * x

def tahn(x):
    return 2 / (1 + np.exp(-2*x)) - 1

def softmax(x):
    return np.array([np.exp(it) / np.sum(np.exp(it)) for it in x])

In [19]:
# Производные ф-ий активации #
def sigmoid_d(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

def relu_d(x):
    arr = x > 0
    return arr.astype(float)
    
def tahn_d(x):
    return 1 - tahn(x)**2
    
def softmax_d(x):
    return np.array([it * (1 - it) for it in x])

In [20]:
# Класс нейронки #
class NeuralNetwork:
    def __init__(self, x, y_class, y_regres, n1_count, n2_count):
        # Входные данные #
        self.input         = x
        self.y_class       = y_class
        self.y_regres      = y_regres
        self.output_class  = np.zeros(len(y_class))
        self.output_regres = np.zeros(len(y_regres))
        
        # Веса #
        # 1 - кол-во входов, 2 - кол-во нейронов #
        # Веса первого слоя #
        self.w1 = np.random.randn(self.input.shape[1], n1_count) / np.sqrt(n1_count)
        # Веса второго слоя #
        self.w2 = np.random.randn(n1_count, n2_count) / np.sqrt(n2_count)
        # Веса выходных слоев #
        self.w_out_class  = np.random.randn(n2_count, 1)
        self.w_out_regres = np.random.randn(n2_count, 1)
        
        # Прямой проход #
    def FeedForward(self, x):
        self.k1           = sigmoid(np.dot(x, self.w1))
        self.k2_class     = sigmoid(np.dot(self.k1, self.w2))
        self.k2_regres    = sigmoid(np.dot(self.k1, self.w2))
        self.k_out_class  = sigmoid(np.dot(self.k2_class, self.w_out_class))
        self.k_out_regres = relu_d(np.dot(self.k2_regres, self.w_out_regres))
        
        return self.k_out_class, self.k_out_regres
        
        # Обратный проход #
    def BackProp(self, x, y_class, y_regres):
        # Упрощения #
        d_pred_class  = 2 * (y_class - self.k_out_class)
        d_pred_regres = 2 * (y_regres - self.k_out_regres)
        d_out_class   = d_pred_class * sigmoid_d(self.output_class) # Для выходного слоя классификации #
        d_out_regres  = d_pred_regres * relu(self.output_regres) # Для выходного слоя регрессии #
        easy_1 = np.dot(d_out_class, self.w_out_class.T)*sigmoid_d(self.k2_class)
        easy_2 = np.dot(d_out_regres, self.w_out_regres.T)*sigmoid_d(self.k2_regres)
        
        # Тут могут быть ошибки #
        d_w_out_class  = np.dot(self.k2_class.T, d_out_class) # [64,1]
        d_w_out_regres = np.dot(self.k2_regres.T, d_out_regres) # [64,1]
        d_w2_class     = np.dot(self.k1.T, easy_1)
        d_w2_regres    = np.dot(self.k1.T, easy_2)
        d_w1_class     = np.dot(x.T, np.dot(easy_1, self.w2.T)*sigmoid_d(self.k1))
        d_w1_regres    = np.dot(x.T, np.dot(easy_2, self.w2.T)*sigmoid_d(self.k1))
        
        # -------------Место для тестов------------------ #
        #print(easy_1[:1])
        #print(easy_2[:1])
        #print("d_out_class " + str(d_out_class.shape))
        #print("d_w_out_class " + str(d_w_out_class.shape))
        #print("easy " + str(easy.shape))
        # -------------Место для тестов------------------ #
        
        # Обновление весов #
        self.w1           += d_w1_class
        self.w1           += d_w1_regres
        self.w2           += d_w2_class
        self.w2           += d_w2_regres
        self.w_out_class  += d_w_out_class
        self.w_out_regres += d_w_out_regres
        
        # Тренировка #
    def train(self, x, y_class, y_regres):
        self.output_class, self.output_regres = self.FeedForward(x)
        self.BackProp(x, y_class, y_regres)
        

In [38]:
# Кол-во нейронов в скрытых слоях #
n1_count = int(input("Укажите кол-во нейронов в 1-м слое: "))
n2_count = int(input("Укажите кол-во нейронов во 2-м слое: "))

# Ограничение, ну так, просто #
if n1_count <= 0:
    n1_count = 1
    
if n2_count <= 0:
    n2_count = 1

# Объявление класса #
nn = NeuralNetwork(X_train_class_scaled, Y_train_class, Y_train_regres, n1_count, n2_count)
#nn = NeuralNetwork(X_train_class_iris_scaled, Y_train_class_iris, Y_train_regres_iris, n1_count, n2_count)

# Вывод данных #
for i in range(1000):
    if i == 0 or i == 500 or i == 999:
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X_train_class_scaled))
        print ("Actual Class Output: \n" + str(Y_train_class[:10]))
        print ("Actual Regres Output: \n" + str(Y_train_regres[:10]))
        print ("Predicted Class Output: \n" + str(nn.output_class[:10]))
        print ("Predicted Regres Output: \n" + str(nn.output_regres[:10]))
        print ("Class Loss: \n" + str(np.mean(np.square(Y_train_class - nn.output_class)))) # mean sum squared loss
        print ("Regres Loss: \n" + str(np.mean(np.square(Y_train_regres - nn.output_regres)))) # mean sum squared loss
        print ("\n")
    nn.train(X_train_class_scaled, Y_train_class, Y_train_regres)
    #nn.train(X_train_class_iris_scaled, Y_train_class_iris, Y_train_regres_iris)

Укажите кол-во нейронов в 1-м слое: 128
Укажите кол-во нейронов во 2-м слое: 64
for iteration # 0

Actual Class Output: 
[[1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]]
Actual Regres Output: 
[[0.06915]
 [0.1109 ]
 [0.07247]
 [0.09075]
 [0.05865]
 [0.09772]
 [0.0681 ]
 [0.07873]
 [0.05974]
 [0.07409]]
Predicted Class Output: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted Regres Output: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class Loss: 
0.6291079812206573
Regres Loss: 
0.007254909534507043




c:\users\godoftea\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


for iteration # 500

Actual Class Output: 
[[1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]]
Actual Regres Output: 
[[0.06915]
 [0.1109 ]
 [0.07247]
 [0.09075]
 [0.05865]
 [0.09772]
 [0.0681 ]
 [0.07873]
 [0.05974]
 [0.07409]]
Predicted Class Output: 
[[1.00000000e+000]
 [1.66069701e-143]
 [1.00000000e+000]
 [1.66069701e-143]
 [1.66069701e-143]
 [1.66069701e-143]
 [1.00000000e+000]
 [1.66069701e-143]
 [1.00000000e+000]
 [1.00000000e+000]]
Predicted Regres Output: 
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Class Loss: 
0.023474178403755867
Regres Loss: 
0.007254909534507042


for iteration # 999

Actual Class Output: 
[[1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]]
Actual Regres Output: 
[[0.06915]
 [0.1109 ]
 [0.07247]
 [0.09075]
 [0.05865]
 [0.09772]
 [0.0681 ]
 [0.07873]
 [0.05974]
 [0.07409]]
Predicted Class Output: 
[[1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]]
Predicted Regres Output: 
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.